In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import re
import numpy as np
import pickle
import os

class FileData(object):
    def __init__(self, path):
        self.path = path
        with open(path, encoding='utf-16') as f:
          self.data = f.read()
          #print(self.data)

ABSOLUTE_PATH = r"/content/drive/MyDrive/vnese-spell-correction-lstm/Train_Full"

c_tri =  "/Chinh tri Xa hoi"

doi_song = "/Doi song"

khoa_hoc = "/Khoa hoc"

kinh_doanh = "/Kinh doanh"

p_luat = "/Phap luat"

suc_khoe = "/Suc khoe"

the_gioi = "/The gioi"

the_thao = "/The thao"

van_hoa = "/Van hoa"

vi_tinh = "/Vi tinh"
corpus = [c_tri, doi_song, khoa_hoc, kinh_doanh, p_luat, suc_khoe, the_gioi, the_thao, van_hoa, vi_tinh]

#Extract folder path
for folder_path in range(len(corpus)):
    corpus[folder_path] = ABSOLUTE_PATH + corpus[folder_path]

file_list = []

#Extracting text from corpus
for folder_path in corpus:
    count = 0
    for name in os.listdir(folder_path):
        count +=1
        if count == 1500:
          break
        path = os.path.join(folder_path, name)
        if not os.path.isfile(path):
            continue
        file = FileData(path)
        file_list.append( file.data )

print('Corpus length: ', len(file_list))

#Save extracted corpus as Pickle file
path_corpus = r"/content/drive/MyDrive/vnese-spell-correction-lstm/train_corpus.pkl"

with open(path_corpus, 'wb') as pickle_file:
    pickle.dump(file_list, pickle_file)

Corpus length:  1928


In [ ]:
!pip install unidecode

In [ ]:
import re
import numpy as np
import pickle
from unidecode import unidecode
import itertools
from nltk import ngrams
from tqdm import tqdm

path_corpus = r"/content/drive/MyDrive/vnese-spell-correction-lstm/train_corpus.pkl"

with open(path_corpus, "rb") as f:
    data = pickle.load(f)

alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'

#Extracting sentence from corpus
def latin_extract(data):

    # extract Latin- characters only
    latin_extract_data=[]
    # duyet qua tung van ban
    for i in data:
      if i == 1:
        break
      # thay the xuong dong la dau cham ket thuc
      i=i.replace("\n",".")
      # tach van ban theo dau cham ket thuc
      sentences=i.split(".")
      for j in sentences:
          if len(j.split()) > 2 and re.match(alphabet, j.lower()):

              latin_extract_data.append(j)

    return latin_extract_data

training_data = latin_extract(data)
i = 100
#Listing all typos, regional dialects
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
letters2=list("abcdefghijklmnopqrstuvwxyz")

typo={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}

# dia phuong
region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}

# nguyen am
vowel=list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")

# viet tat
acronym={"không":"ko"," anh":" a","em":"e","biết":"bít","giờ":"h","gì":"j","muốn":"mún","học":"hok","yêu":"iu",
         "chồng":"ck","vợ":"vk"," ông":" ô","được":"đc","tôi":"t",
         "Không":"Ko"," Anh":" A","Em":"E","Biết":"Bít","Giờ":"H","Gì":"J","Muốn":"Mún","Học":"Hok","Yêu":"Iu",
         "Chồng":"Ck","Vợ":"Vk"," Ông":" Ô","Được":"Đc","Tôi":"T",}

# teencode
teen={"ch":"ck","ph":"f","th":"tk","nh":"nk",
      "Ch":"Ck","Ph":"F","Th":"Tk","Nh":"Nk"}

# function for adding mistake( noise)
def teen_code(sentence,pivot):
    random = np.random.uniform(0,1,1)[0]
    new_sentence=str(sentence)
    if random>pivot:
        for word in acronym.keys():
            if re.search(word, new_sentence):
                random2 = np.random.uniform(0,1,1)[0]
                if random2 <0.5:
                    new_sentence=new_sentence.replace(word,acronym[word])
        for word in teen.keys():
            if re.search(word, new_sentence):
                random3 = np.random.uniform(0,1,1)[0]
                if random3 <0.05:
                    new_sentence=new_sentence.replace(word,teen[word])
        return new_sentence
    else:
        return sentence


def add_noise(sentence, pivot1,pivot2):
    sentence=teen_code(sentence,0.5)
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence+=sentence[i]
        else:
            random = np.random.uniform(0,1,1)[0]
            if random < pivot1:
                noisy_sentence+=(sentence[i])
            elif random<pivot2:
                if sentence[i] in typo.keys() and sentence[i] in region.keys():
                    random2=np.random.uniform(0,1,1)[0]
                    if random2<=0.4:
                        noisy_sentence+=typo[sentence[i]]
                    elif random2<0.8:
                        noisy_sentence+=region[sentence[i]]
                    elif random2<0.95 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in typo.keys():
                    random3=np.random.uniform(0,1,1)[0]
                    if random3<=0.6:
                        noisy_sentence+=typo[sentence[i]]
                    elif random3<0.9 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in region.keys():
                    random4=np.random.uniform(0,1,1)[0]
                    if random4<=0.6:
                        noisy_sentence+=region[sentence[i]]
                    elif random4<0.85 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif i<len(sentence)-1 :
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random5=np.random.uniform(0,1,1)[0]
                        if random5<=0.9:
                            noisy_sentence+=region2[sentence[i]]
                        else:
                            noisy_sentence+=sentence[i]
                    else:
                        noisy_sentence+=sentence[i]

            else:
                new_random = np.random.uniform(0,1,1)[0]
                if new_random <=0.33:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence+=(sentence[i+1])
                        noisy_sentence+=(sentence[i])
                        i += 1
                elif new_random <= 0.66:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence+=random_letter
                else:
                    pass

        i += 1
    return noisy_sentence

def extract_phrases(text):
    return re.findall(r'\w[\w ]+', text)

def _extract_phrases(data):
    phrases = itertools.chain.from_iterable(extract_phrases(text) for text in data)
    phrases = [p.strip() for p in phrases if len(p.split()) > 1]

    return phrases

phrases = _extract_phrases(training_data)

#Generate Bi-gram

#A Vietnamese word do not contain more than 7 characters, so an bi-gram do not have more than 15 characters
NGRAM = 2
MAXLEN = 40

def gen_ngrams(words, n=2):
    return ngrams(words.split(), n)
#Mục đích: Tạo danh sách các bi-grams từ danh sách các cụm từ (phrases) và lọc ra các bi-grams thỏa mãn các điều kiện.
def generate_bi_grams(phrases):
    list_ngrams = []
    for p in tqdm(phrases):

      # neu khong nham trong bang chu cai thi bo qua
      if not re.match(alphabet, p.lower()):
        continue

      # tach p thanh cac bi gram
      for ngr in gen_ngrams(p, NGRAM):
        if len(" ".join(ngr)) < MAXLEN:
          list_ngrams.append(" ".join(ngr))

    return list_ngrams

list_ngrams = generate_bi_grams(phrases)

print(len(list_ngrams))

alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']

#Chuyển đổi một đoạn văn bản thành dạng mã hóa one-hot có độ dài cố định (MAXLEN).
#Đầu ra: Mảng 2D one-hot encoding của văn bản
def encoder_data(text, maxlen=MAXLEN):
        #print("Maxlen", maxlen)
        text = "\x00" + text
        #print("text", text)
        x = np.zeros((maxlen, len(alphabet)))
        #print("X ban dau", x)
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x
#Mục đích: Giải mã dữ liệu one-hot encoding trở lại thành văn bản.
#Đầu ra: Văn bản đã được giải mã.
def decoder_data(x):
    x = x.argmax(axis=-1)
    #print("x hien tai", x)
    dem = ''.join(alphabet[i] for i in x)
    #print("Do dai cau van", len(dem))

    return dem


100%|██████████| 64294/64294 [00:00<00:00, 109226.43it/s]

614698


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense,LSTM, Bidirectional
from keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
# import preprocessing
import os
# from preprocessing import MAXLEN, alphabet

encoder = LSTM(256,input_shape=(MAXLEN, len(alphabet)), return_sequences=True)

decoder = Bidirectional(LSTM(256, return_sequences=True, dropout=0.2))

model=Sequential()
model.add(encoder)
model.add(decoder)
model.add(TimeDistributed(Dense(256)))
model.add(Activation('relu'))
model.add(TimeDistributed(Dense(len(alphabet))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

model.summary()

#Spliting training data
train_data, valid_data = train_test_split(list_ngrams, test_size=0.2, random_state=42)

# we have to use data- generation medthod cause this dataset is too large to fit into memory
BATCH_SIZE = 512
def generate_data(data, batch_size):
    cur_index = 0
    while True:
        x, y = [], []
        for i in range(batch_size):
            y.append(encoder_data(data[cur_index]))
            x.append(encoder_data(add_noise(data[cur_index],0.94,0.985)))
            cur_index += 1
            if cur_index > len(data)-1:
                cur_index = 0
        yield np.array(x), np.array(y)

train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
validation_generator = generate_data(valid_data, batch_size=BATCH_SIZE)

# train the model and save to the Model folder
checkpointer = ModelCheckpoint(filepath=os.path.join('/content/drive/MyDrive/vnese-spell-correction-lstm/spelling_ver1.h5'), save_best_only=True, verbose=1)

model.fit( train_generator, steps_per_epoch=len(train_data)//BATCH_SIZE, epochs=5,
                    validation_data=validation_generator, validation_steps=len(valid_data)//BATCH_SIZE,
                    callbacks=[checkpointer] )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40, 256)           466944    
                                                                 
 bidirectional (Bidirection  (None, 40, 512)           1050624   
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 40, 256)           131328    
 ributed)                                                        
                                                                 
 activation (Activation)     (None, 40, 256)           0         
                                                                 
 time_distributed_1 (TimeDi  (None, 40, 199)           51143     
 stributed)                                                      
                                                        

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


960/960 [==============================] - 160s 148ms/step - loss: 0.3517 - accuracy: 0.9271 - val_loss: 0.0729 - val_accuracy: 0.9855
Epoch 2/5
960/960 [==============================] - ETA: 0s - loss: 0.0595 - accuracy: 0.9874
Epoch 2: val_loss improved from 0.07287 to 0.04660, saving model to /content/drive/MyDrive/vnese-spell-correction-lstm/spelling_ver1.h5
960/960 [==============================] - 152s 159ms/step - loss: 0.0595 - accuracy: 0.9874 - val_loss: 0.0466 - val_accuracy: 0.9895
Epoch 3/5
960/960 [==============================] - ETA: 0s - loss: 0.0416 - accuracy: 0.9906
Epoch 3: val_loss improved from 0.04660 to 0.03575, saving model to /content/drive/MyDrive/vnese-spell-correction-lstm/spelling_ver1.h5
960/960 [==============================] - 137s 143ms/step - loss: 0.0416 - accuracy: 0.9906 - val_loss: 0.0358 - val_accuracy: 0.9917
Epoch 4/5
960/960 [==============================] - ETA: 0s - loss: 0.0340 - accuracy: 0.9921
Epoch 4: val_loss improved from 0.0357

In [ ]:
print(add_noise("chúc mừng năm mới 2024",0.94,0.985));

chúc mừng năm mới 2024


In [ ]:
from collections import Counter
from keras.models import load_model
from nltk import ngrams,word_tokenize
import numpy as np
import re
import string

model = load_model('/content/drive/MyDrive/vnese-spell-correction-lstm/spelling_ver1.h5')

NGRAM=2
MAXLEN=40
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
accepted_char=list((string.digits + ''.join(letters)))

def call(sentence):
    def encoder_data(text, maxlen=MAXLEN):
            text = "\x00" + text
            x = np.zeros((maxlen, len(alphabet)))
            for i, c in enumerate(text[:maxlen]):
                x[i, alphabet.index(c)] = 1
            if i < maxlen - 1:
              for j in range(i+1, maxlen):
                x[j, 0] = 1
            return x

    def decoder_data(x):
        x = x.argmax(axis=-1)
        return ''.join(alphabet[i] for i in x)

    def nltk_ngrams(words, n=2):
        return ngrams(words.split(), n)

    def guess(ngram):
        text = ' '.join(ngram)
        preds = model.predict(np.array([encoder_data(text)]), verbose=0)
        return decoder_data(preds[0]).strip('\x00')

    def correct(sentence):
        for i in sentence:
            if i not in accepted_char:
                sentence=sentence.replace(i," ")
        ngrams = list(nltk_ngrams(sentence, n=NGRAM))
        guessed_ngrams = list(guess(ngram) for ngram in ngrams)

        print("N gram", ngrams)
        print("guess", guessed_ngrams)

        candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
        for nid, ngram in (enumerate(guessed_ngrams)):
            for wid, word in (enumerate(re.split(' +', ngram))):
                candidates[nid + wid].update([word])

        output = ' '.join(c.most_common(1)[0][0] for c in candidates)
        return output

    guess = correct(sentence)

    return guess

sentence = "Tôi là sinh vien trường đại họ cPhenikaa taji Hà Dông"
# sentence = 'Tôi phông iu thích hok ngôn ngữ mới'
# sentence = 'tôi teen laf Tô Văn Tus, toi ddang hojc lớp 5'
# sentence = "chusc mừg năm mới 2024"

guess = call(sentence)
print(guess)

N gram [('Tôi', 'là'), ('là', 'sinh'), ('sinh', 'vien'), ('vien', 'trường'), ('trường', 'đại'), ('đại', 'họ'), ('họ', 'cPhenikaa'), ('cPhenikaa', 'taji'), ('taji', 'Hà'), ('Hà', 'Dông')]
guess ['Tôi là', 'là sinh', 'sinh viện', 'viện trường', 'trường đại', 'đại họ', 'họ cPhenikaa', 'cPhenikaa tại', 'tại Hà', 'Hà Dông']
Tôi là sinh viện trường đại họ cPhenikaa tại Hà Dông
